把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling，在其他语言中也被称之为serialization，marshalling，flattening等等，都是一个意思。

序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上。

反过来，把变量内容从序列化的对象重新读到内存里称之为反序列化，即unpickling。

In [3]:
import pickle
d = dict(name='Bob', age=20, score=88)
x = pickle.dumps(d)
print(x)

b'\x80\x03}q\x00(X\x04\x00\x00\x00nameq\x01X\x03\x00\x00\x00Bobq\x02X\x03\x00\x00\x00ageq\x03K\x14X\x05\x00\x00\x00scoreq\x04KXu.'


pickle.dumps()方法把任意对象序列化成一个bytes，然后，就可以把这个bytes写入文件。

另一个方法pickle.dump()直接把对象序列化后写入一个file-like Object。

In [7]:
d = dict(name='Bob', age=20, score=88)
with open('dump.txt', 'wb') as f:
    pickle.dump(d, f)

当我们要把对象从磁盘读到内存时，可以先把内容读到一个bytes，然后用pickle.loads()方法反序列化出对象，也可以直接用pickle.load()方法从一个file-like Object中直接反序列化出对象。

In [12]:
with open('dump.txt', 'rb') as f:
    d = pickle.load(f)    #内容相同的一个新对象
print(d)

{'name': 'Bob', 'age': 20, 'score': 88}


Pickle的问题和所有其他编程语言特有的序列化问题一样，就是它只能用于Python，并且可能不同版本的Python彼此都不兼容，因此，只能用Pickle保存那些不重要的数据，不能成功地反序列化也没关系。

JSON

如果我们要在不同的编程语言之间传递对象，就必须把对象序列化为标准格式，比如XML，但更好的方法是序列化为JSON，因为JSON表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。JSON不仅是标准格式，并且比XML更快，而且可以直接在Web页面中读取，非常方便。

JSON表示的对象就是标准的JavaScript语言的对象，JSON和Python内置的数据类型对应关系：

| JSON类型 | Python类型 |
| :-----: | :------: |
| {} | dict |
| [] | list |
| "string" | str |
| 1234.56 | int或float |
| true/false | True/False |
| null | None |

In [14]:
import json
d = dict(name='Bob', age=20, score=88)
json_str = json.dumps(d)
print(json_str)

{"name": "Bob", "age": 20, "score": 88}


dumps()方法返回一个str，内容就是标准的JSON。类似的，dump()方法可以直接把JSON写入一个file-like Object。

要把JSON反序列化为Python对象，用loads()或者对应的load()方法，前者把JSON的字符串反序列化，后者从file-like Object中读取字符串并反序列化。

In [16]:
json_str = '{"name": "Bob", "age": 20, "score": 88}'
d = json.loads(json_str)
print(type(d))
print(d)

<class 'dict'>
{'name': 'Bob', 'age': 20, 'score': 88}


JSON进阶

Python的dict对象可以直接序列化为JSON的{}，不过，很多时候，我们更喜欢用class表示对象，比如定义Student类，然后序列化。

In [19]:
import json

class Student(object):
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score

def student2dict(std):
    return {
        'name': std.name,
        'age': std.age,
        'score': std.score
    }

s = Student('Bob', 20, 88)
json_str = json.dumps(s, default=student2dict)    #可选参数default：把任意一个对象变成可序列为JSON的对象
print(json_str)

{"name": "Bob", "age": 20, "score": 88}


In [20]:
#把任意class的实例变为dict：
json_str = json.dumps(s, default=lambda obj: obj.__dict__)
print(json_str)

{"name": "Bob", "age": 20, "score": 88}


因为通常class的实例都有一个\__dict\__属性，它就是一个dict，用来存储实例变量。也有少数例外，比如定义了\__slots\__的class。

同样的道理，如果我们要把JSON反序列化为一个Student对象实例，loads()方法首先转换出一个dict对象，然后，我们传入的object_hook函数负责把dict转换为Student实例。

In [23]:
def dict2student(d):
    return Student(d['name'], d['age'], d['score'])

json_str = '{"name": "Bob", "age": 20, "score": 88}'
s1 = json.loads(json_str, object_hook=dict2student)
print(s1)


In [24]:
#练习：对中文进行JSON序列化时，json.dumps()提供了一个ensure_ascii参数，观察该参数对结果的影响
import json 

obj = dict(name='小明', age=20)
s = json.dumps(obj, ensure_ascii=True)
print(s)

{"name": "\u5c0f\u660e", "age": 20}


In [26]:
obj = dict(name='小明', age=20)
s = json.dumps(obj, ensure_ascii=False)
print(s)

{"name": "小明", "age": 20}


In [7]:
#json字符串转化为任意类
import json
def MyJsonLoads(cls, jsonStr):
    #字符串转换为字典
    jsonDict = json.loads(jsonStr)
    className = str(type(cls))[str(type(cls)).find('.') + 1:-2]
    newObj = eval(className + '()')
    #动态添加属性
    for key in jsonDict:
        setattr(newObj, key, jsonDict[key])
    return newObj

class Student(object):
    pass

#用的时候将一个该类型的对象和对应的json字符串传进去就好了
s = MyJsonLoads(Student(),'{"name":"123","age":16}')